In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pk
import re 

path = "d:/Facultad/Tesis/"
path_git = "d:/Git_Proyects/Tesis/Base_Twitter/Tweets_August/"
base_nac = pd.read_csv(path+'Corpus_medios_nac.csv')
#Tweets_mayo = pd.read_csv(path_git+'Tweets_August_solo_medios.csv')
Tweets_mayo = pd.read_csv(path_git+'Tweets_August_solo_medios.csv')
Base_sin_scrap = pd.read_csv(path+'2019_news_content.csv', delimiter =';')

C:\Users\Ale\AppData\Local\Temp\ipykernel_2464\642691954.py:9: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  base_nac = pd.read_csv(path+'Corpus_medios_nac.csv')


In [3]:
Tweets_unique = Tweets_mayo.drop_duplicates(subset='urls')

patron = r'www\.(.*?)\.com'
data = []
for i in range(len(Tweets_unique)):
    try:
        resultado = re.search(patron, Tweets_unique['urls'][i])
        data.append(resultado.group(1))
    except:
        pass

In [4]:
print(data.count('infobae'))
print(set(data))

132
{'elliberal', 'lavoz', 'diarioregistrado', 'eldestapeweb', 'lanacion', 'ellitoral', 'cronista', 'elciudadanoweb', 'laizquierdadiario', 'elancasti', 'diariamenteneuquen', 'xn--lamaanaonline-lkb', 'pagina12', 'infobae', 'ambito', 'diarioprensa', 'eldia', 'perfil', 'clarin', 'lagaceta', 'telam', 'diarioelzondasj', 'corrienteshoy', 'elesquiu'}


In [5]:
def replace(x):
    x = str(x)
    x = x.replace('”', '"')
    x = x.replace('“', '"')
    x = x.replace('”', '"')
    x = x.replace('\x93', '"')
    x = x.replace('\x94', '"')
    
    return x

def extract_quotes(x):
    return re.findall('"([^"]*)"', x)

def words_length(x):
    return len(x.split(" "))

def pre_process_quoted(df1):
    df = df1.copy()
#    df['nota'] = df['nota'].apply(replace)
    df.loc[:, 'nota'] = df['nota'].apply(replace)
    #df['citas']  = df['nota'].apply(extract_quotes)
    df.loc[:, 'citas'] = df['nota'].apply(extract_quotes)

    df_quoutes2 = pd.DataFrame({'Cita': df['citas'].iloc[1:], 'Link': df['link'].iloc[1:], 'Nota': df['nota'].iloc[1:]})
    #df_quoutes2 = pd.DataFrame({'Fecha': df['fecha'].iloc[1:], 'Hora': df['hora'].iloc[1:], 'Cita': df['citas'].iloc[1:], 'Link': df['link'].iloc[1:], 'Nota': df['nota'].iloc[1:]})
    df_quoutes = df_quoutes2.explode('Cita') # Cada cita sea una fila distinta
    df_quoutes = df_quoutes.dropna(subset=['Cita']) # Elimino la filas que no haya quotes

    #df_quoutes['Citas'] = df_quoutes['Citas'].apply(replace)

    df_quoutes = df_quoutes.reset_index()
    df_quoutes = df_quoutes.drop(['index'], axis = 1)
    df_quoutes['Cant_Palabras'] = df_quoutes['Cita'].apply(words_length) # genero columna que cuente la cantidad de plabaras
    df_quoutes_pf = df_quoutes[df_quoutes['Cant_Palabras'] >= 4]
    df_quoutes_pf = df_quoutes_pf[df_quoutes_pf['Cant_Palabras'] < 60]
    df_quoutes_pf = df_quoutes_pf.reset_index()
    df_quoutes_pf = df_quoutes_pf.drop(['index'], axis = 1)
    return df_quoutes_pf

In [6]:
Tweets_mayo.sort_values(by=['link_nacionales'])
Tweets_unique = Tweets_mayo.drop_duplicates(subset='urls')
Tweets_to_scrap = Tweets_unique[Tweets_unique['link_nacionales'] == 2]
#df_urls_unique[df_urls_unique['link_nacionales'] ==1].count()

In [7]:
Tweets_mayo[Tweets_mayo['link_nacionales'] == 1]
base_nac['medios_comp'] = base_nac['link'].isin(Tweets_mayo['urls'])
base_lucio = base_nac[base_nac['medios_comp']==True]
extract_quotes_1  = pre_process_quoted(base_lucio)



Tweets_to_scrap = Tweets_unique[Tweets_unique['link_nacionales'] == 2]
Base_sin_scrap.rename(columns={'url':'urls'}, inplace=True)
Tweets_to_scrap = Tweets_to_scrap.merge(Base_sin_scrap, how= 'left', on=  'urls')
Tweets_to_scrap.rename(columns={'texto': 'nota','urls':'link'}, inplace=True)
Tweets_to_scrap
extract_quotes_2 = pre_process_quoted(Tweets_to_scrap)
len(extract_quotes_1), len(extract_quotes_2)

(18006, 7689)

In [7]:
citas = list(extract_quotes_1['Cita']) + list(extract_quotes_2['Cita'])

In [8]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')  

C:\Users\Ale\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
#citas_vect = model.encode(citas, convert_to_tensor=True)
#pk.dump(citas_vect, open(path_git+'emb_october_1_2.pk', 'wb'))

In [40]:
citas_vect  = pk.load(open(path_git+'emb_october_1_2.pk', 'rb'))

In [41]:
citas_df = pd.DataFrame({'Cita':citas})

In [42]:
clusters = util.community_detection(citas_vect,threshold=0.8, min_community_size = 1)  
cluste_map = {citas[j]: str(i+1) for i, clus in enumerate(clusters) for j in clus}
citas_df['id_cluster'] = citas_df['Cita'].map(cluste_map)

In [44]:
for cit in citas_df[citas_df['id_cluster'] == '2']['Cita']:
    print(cit)

hay más gente que se dedica a vender droga porque se quedó sin laburo
ahora hay gente que se dedica a vender droga porque se quedó sin laburo
ahora hay gente que se dedica a vender droga porque se quedó sin laburo
Hay más gente que se dedica a vender droga porque se quedó sin laburo
Hay gente que se dedica a vender droga porque se quedó sin laburo
hay gente que se dedica a vender droga porque se quedó sin laburo
hay gente que se dedica a vender droga porque se quedó sin laburo
hay más gente que se dedica a vender droga porque se quedó sin laburo
Ahora hay gente que se dedica a vender droga porque se quedó sin laburo
hay gente que se dedica a vender droga porque se quedó sin laburo
hay gente que se dedica a vender droga porque se quedó sin laburo
hay gente que se dedica a vender droga porque se quedó sin laburo
hay gente que se dedica a vender droga porque se quedó sin laburo
con la pérdida de empleo hay gente que se dedica a vender droga porque se quedó sin laburo, ob

In [45]:
frases_medias = []
for i in range(1500, 3000):
    cit_mean = []
    for cit in citas_df[citas_df['id_cluster'] == str(i)]['Cita']:
        cit_mean.append(cit.lower())
    try: 
        contador = {}
        for elemento in cit_mean:
            if elemento in contador:
                contador[elemento] +=1
            else:
                contador[elemento] = 1

        elemento_mas_comun = max(contador, key = contador.get)
        frases_medias.append(elemento_mas_comun)
    except:
        print(i)

In [46]:
file = open(path_git+'citas/muchas_citas_october_medias_1500_3000_T_L.txt', 'w', encoding='utf8')
for cita in frases_medias:
    file.write(cita+'\n')
file.close()